In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime

In [2]:
site = 'https://www.oddsshark.com'
lg_tks = {}
lg_futs = {}
lg_tks['UFC'] = requests.get(
    'https://io.oddsshark.com/ticker/ufc', 
    headers = {
        'referer': site
    }
)

for league, active in lg_tks['UFC'].json()['leagues'].items():
    lg_futs[league.upper()] = BeautifulSoup(requests.get(f"{site}/{league}/odds/futures").content, 'html.parser')
    if active==True:
        lg_tks[league.upper()] = requests.get(
            'https://io.oddsshark.com/ticker/' + league, 
            headers = {
                'referer': site
            }
        ).json()['matchups']

In [ ]:
yesterday = date.today() - timedelta(days=1)
today = date.today()
tomorrow = date.today() + timedelta(days=1) 

In [27]:
lg_tks['NHL']

[{'type': 'date',
  'date': {'day': '11', 'fullday': 'Sunday', 'month': 'Apr'},
  'current': False},
 {'type': 'matchup',
  'event_id': '1367221',
  'event_date': '2021-04-11 14:00',
  'away_short_name': 'BUF',
  'home_short_name': 'PHI',
  'away_name': 'Buffalo',
  'home_name': 'Philadelphia',
  'home_votes': 71,
  'away_votes': 29,
  'over_votes': 73,
  'under_votes': 27,
  'total': '5.5',
  'away_odds': '180',
  'home_odds': '-220',
  'status': 'FINAL',
  'away_score': '5',
  'home_score': '3',
  'trending_total': 8,
  'tv_station': 'NHL',
  'matchup_link': '/nhl/buffalo-philadelphia-odds-april-11-2021-1367221'},
 {'type': 'matchup',
  'event_id': '1369721',
  'event_date': '2021-04-11 16:00',
  'away_short_name': 'ARI',
  'home_short_name': 'VGK',
  'away_name': 'Arizona',
  'home_name': 'Vegas',
  'home_votes': 71,
  'away_votes': 29,
  'over_votes': 37,
  'under_votes': 63,
  'total': '5.5',
  'away_odds': '210',
  'home_odds': '-250',
  'status': 'FINAL',
  'away_score': '0',
  

In [ ]:
def simple_odds(odds):
    for game in odds:
        if game['type'] == 'date':
            print('--', game['date']['fullday'], game['date']['month'], game['date']['day'])
        if game["type"] == "matchup":
            if game['status'].startswith('FINAL'):
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_score'], '| font=Courier')
                print('----', game['home_short_name'], game['home_score'], '| font=Courier')
            else:
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_odds'] if game['away_odds'].startswith('-') else '+' + game['away_odds'], '| font=Courier')
                print('----', game['home_short_name'], game['home_odds'] if game['home_odds'].startswith('-') else '+' + game['home_odds'], '| font=Courier')

In [ ]:
for sport, odds in lg_tks.items():
    if sport != 'UFC':
        print(sport)
        simple_odds(odds)
        print('---')

In [ ]:
lg_tks['UFC']

In [ ]:
print('UFC')
for fight in lg_tks['UFC']:
    if fight['type'] == 'event':
        print('--', fight['event'])
    if fight["type"] == 'matchup':
        if not fight['status']:
            print('----', fight['event_date'][11:])
            print('----', fight['away_name'], fight['away_odds'] if fight['away_odds'].startswith('-') else '+' + fight['away_odds'], '| font=Courier')
            print('----', fight['home_name'], fight['home_odds'] if fight['home_odds'].startswith('-') else '+' + fight['home_odds'], '| font=Courier')
        else:
            print('----', 'FINAL')
            print('----', fight['away_name'], fight['status'] if fight['away_name'] == fight['winner'] else '', '| font=Courier')
            print('----', fight['home_name'], fight['status'] if fight['home_name'] == fight['winner'] else '', '| font=Courier')

In [ ]:
print(soup.prettify())

In [ ]:
opening_odds = soup.find_all('div', class_='op-item op-future-item')
opening_list = []
for elem in opening_odds:
    opening_list.append(elem.get_text())

In [ ]:
current_odds = soup.find_all('div', class_='op-item op-future-item no-vegas')
current_list = []
for elem in current_odds:
    current_list.append(elem.get_text())

In [ ]:
left_column = soup.find('div', class_='op-team-data-wrapper futures')
left_column = futures_teams.find_all(['div','span'], class_=['align-bottom','op-team hockey op-odd','op-team hockey op-even'])
team_list =[]
for elem in left_column:
    team_list.append(elem.get_text().strip())

In [3]:
def process_odds_section(soup, html_type, html_class):
    data = soup.find_all(html_type, class_=html_class)
    init_list = []
    daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    for elem in data:
        if not elem.get_text().startswith(tuple(daysofweek)):
            init_list.append(elem.get_text().strip())
    return init_list

In [4]:
# works for all leagues for current_odds
opening_odds = process_odds_section(lg_futs['MLB'], 'div', 'op-item op-future-item')
current_odds = process_odds_section(lg_futs['MLB'], 'div', 'op-item op-future-item no-vegas')

In [7]:
team_list = process_odds_section(lg_futs['MLB'], ['div','span'], ['align-bottom','op-team baseball op-odd','op-team baseball op-even'])

In [55]:
future_odds_list = []
for a, b, c in zip(*[iter(current_odds)]*3): future_odds_list.append([a,b,c])

In [5]:
def create_futures_data(teams, opening, currents):
    
    futures_trios = []
    for a, b, c in zip(*[iter(currents)]*3):
        futures_trios.append([a,b,c])
    
    rows = []
    odds_inc = 0
    sports = ['MLB','NFL','NHL','NBA','College Basketball','College Football']
    for team in teams:
        if not team.startswith(tuple(sports)):
            rows.append([team,
                         opening[odds_inc],
                         futures_trios[odds_inc][0],
                         futures_trios[odds_inc][1],
                         futures_trios[odds_inc][2]])
            odds_inc += 1
        else:
            rows.append([team])
    return rows

In [8]:
all_odds = create_futures_data(team_list, opening_odds, current_odds)

In [9]:
for odds in all_odds:
    if len(odds) > 1:
        print(odds[0], odds[1], odds[2], odds[3], odds[4])
    else:
        print(odds[0])

MLB - World Series Championship 2021
Arizona +6600 +20000 +20000 +20000
Atlanta +1100 +1200 +1400 +1400
Baltimore +8000 +10000 +10000 +10000
Boston +5000 +3500 +3500 +3500
Chi Cubs +2500 +5000 +5000 +5000
Chi White Sox +1200 +1200 +1200 +1200
Cincinnati +2000 +3500 +3300 +3300
Cleveland +1800 +5500 +4000 +4000
Colorado +8000 +25000 +25000 +25000
Detroit +8000 +15000 +15000 +15000
Houston +1600 +1600 +1400 +1400
Kansas City +10000 +8000 +8000 +8000
LA Angels +4000 +2200 +2200 +2200
LA Dodgers +400 +325 +300 +300
Miami +5000 +10000 +10000 +10000
Milwaukee +5000 +4000 +4000 +4000
Minnesota +1400 +1800 +1800 +1800
NY Mets +3000 +1200 +1200 +1200
NY Yankees +500 +650 +675 +675
Oakland +1800 +4000 +4000 +4000
Philadelphia +3300 +3000 +4000 +4000
Pittsburgh +25000 +40000 +40000 +40000
San Diego +900 +750 +800 +800
San Francisco +6600 +7000 +10000 +10000
Seattle +8000 +10000 +8000 +8000
St. Louis +3300 +2500 +2500 +2500
Tampa Bay +1100 +3300 +2800 +2800
Texas +10000 +17500 +17500 +17500
Toront

In [10]:
for sport in lg_futs:
    opening_odds = process_odds_section(lg_futs[sport], 'div', 'op-item op-future-item')
    current_odds = process_odds_section(lg_futs[sport], 'div', 'op-item op-future-item no-vegas')

UFC
NHL
MLB
NBA
NFL
NCAAF
NCAAB
SOCCER


['+15000',
 '+20000',
 '+20000',
 '+1100',
 '+1200',
 '+1200',
 '+8000',
 '+10000',
 '+10000',
 '+4500',
 '+3500',
 '+3500',
 '+4000',
 '+5000',
 '+5000',
 '+1200',
 '+1200',
 '+1200',
 '+3500',
 '+3300',
 '+3300',
 '+6600',
 '+4000',
 '+4000',
 '+25000',
 '+25000',
 '+25000',
 '+12500',
 '+15000',
 '+15000',
 '+1500',
 '+1400',
 '+1400',
 '+8000',
 '+8000',
 '+8000',
 '+2500',
 '+2200',
 '+2200',
 '+325',
 '+300',
 '+300',
 '+10000',
 '+12500',
 '+12500',
 '+4500',
 '+4000',
 '+4000',
 '+1500',
 '+1800',
 '+1800',
 '+1200',
 '+1400',
 '+1400',
 '+600',
 '+675',
 '+675',
 '+3500',
 '+4000',
 '+4000',
 '+3500',
 '+4000',
 '+4000',
 '+30000',
 '+40000',
 '+40000',
 '+750',
 '+800',
 '+800',
 '+8000',
 '+8000',
 '+8000',
 '+10000',
 '+8000',
 '+8000',
 '+2000',
 '+2500',
 '+2500',
 '+3300',
 '+2800',
 '+2800',
 '+15000',
 '+17500',
 '+17500',
 '+1800',
 '+2200',
 '+2200',
 '+4500',
 '+6000',
 '+6000',
 '+5000',
 '+4000',
 '+4000',
 '+2200',
 '+1800',
 '+1800',
 '+400',
 '+450',
 '+450',
 